In [2]:
%reload_ext autoreload
%autoreload 2

# Language model (text generation, autoregressive model)

This experiment introduce a language model based on Transformers modules. The main idea is to introduce the mask to the delf attention module. Focusing on the process of each elelement of the sequence, the mask avoids to see forward words (avoids taking into account future words of the seqence).
The goal is to predict the next character given a seqence (input is a seqence of characters, output prediction should be the same character sequence shifted to the left)

In [3]:
import warnings
warnings.filterwarnings('ignore')
from _context import src
from src.models.model_utils import device_selection
from src.models.predict_model import GenerationCharacterTransformer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
import torch.distributions as dist

import numpy as np
import random, tqdm, sys, math, gzip


In [9]:
#Params
# NB, the enwik8 data contains tokens from 9 to 240, but well round up to the nearest
# power of two.
NUM_TOKENS = 256

num_classes = 2
embeding_size = 128
transformer_heads = 8
depth = 4
lr = 1e-4
warm = 10000
epochs =100

train_num = int(50e6)
val_num = int(5e6)
test_num = int(5e6)
total_num = train_num + val_num + test_num

# Dataset
Dataset is named enwik8 and contains $10^8$ characters of wikipedia text

In [5]:
# enwik8
# http://mattmahoney.net/dc/enwik8.zip

path = "./dataset/enwik8"
with open(path) as file:
    arr = np.fromstring(file.read(total_num), dtype=np.uint8)
    train_ds, val_ds, test_ds = np.split(arr, [train_num, val_num+ test_num])
    train_ds = torch.from_numpy(train_ds)
    val_ds = torch.from_numpy(val_ds)
    test_ds = torch.from_numpy(test_ds)

In [6]:
train_ds

tensor([ 60, 109, 101,  ...,  61,  61,  61], dtype=torch.uint8)

# Model
Provide a character sequence and the model generates for each character of the seqeuence the next character. hence the last prediction of the putput sequence shuold be the next unseen letter (output is one character shiftted)

<img src="images/transformer_model_generator.svg"  width="500" height="600">

It is necessary that the transformer model can not see further characters in order to perform the attention mechanism. For that reason the weighted matrix is masked with an upper diagonal of $-\inf$ (after softmax it gets a 0)
* Rimind that each row of th eattention weight matrix are the weight of each row sequence vector (each element of the weight vector is the weight of the row sequence vector)
* E.g: the first row of th eweight matrix contains only the fisrt element, the rest are $-inf$. Then, it means that for the first element of the sequence the only vector that counts is the first sequence vector, the others are ignored
<img src="images/mask.svg"  width="500" height="600">

In [21]:
model = GenerationCharacterTransformer(
                            embedding_size=128,
                            transformer_heads=10,
                            depth=5,
                            max_sequence=256, #context
                            token_size=NUM_TOKENS)

In [22]:
#Fast.check
x = torch.rand(8,256)
x = x.long()
o = model(x)
o.size(), o

(torch.Size([8, 256, 256]),
 tensor([[[-6.0941, -5.1164, -6.0955,  ..., -6.9216, -5.9654, -7.1292],
          [-5.7974, -5.1403, -4.1970,  ..., -8.1976, -5.2581, -5.9976],
          [-6.7997, -6.0713, -6.5420,  ..., -6.6436, -5.6532, -6.3152],
          ...,
          [-6.4389, -7.0854, -5.2680,  ..., -7.0332, -5.9197, -7.0326],
          [-6.9156, -5.1362, -5.8684,  ..., -7.5660, -6.7089, -6.5084],
          [-5.8663, -5.7281, -6.1731,  ..., -6.2620, -6.7745, -7.4074]],
 
         [[-6.0941, -5.1164, -6.0955,  ..., -6.9216, -5.9654, -7.1292],
          [-5.7974, -5.1403, -4.1970,  ..., -8.1976, -5.2581, -5.9976],
          [-6.7997, -6.0713, -6.5420,  ..., -6.6436, -5.6532, -6.3152],
          ...,
          [-6.4389, -7.0854, -5.2680,  ..., -7.0332, -5.9197, -7.0326],
          [-6.9156, -5.1362, -5.8684,  ..., -7.5660, -6.7089, -6.5084],
          [-5.8663, -5.7281, -6.1731,  ..., -6.2620, -6.7745, -7.4074]],
 
         [[-6.0941, -5.1164, -6.0955,  ..., -6.9216, -5.9654, -7.1292],


# Train